Задание 0


In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
tr_types_tbl = pd.read_csv('/content/drive/MyDrive/data/tr_types.csv', delimiter=';')
gender_train_tbl = pd.read_csv('/content/drive/MyDrive/data/gender_train.csv')
tr_mcc_codes_tbl = pd.read_csv('/content/drive/MyDrive/data/tr_mcc_codes.csv', delimiter=';')
transactions_tbl = pd.read_csv('/content/drive/MyDrive/data/transactions.csv', nrows=1000000)

Задание 1

In [ ]:
size = 1000
sample = transactions_tbl['tr_type'].sample(size).to_numpy()
descriptions = tr_types_tbl[['tr_type', 'tr_description']].values
filtered = dict(filter(lambda x: 'POS' in x[1] or 'АТМ' in x[1], descriptions))
len(list(filter(lambda x: x in filtered, sample))) / size

0.627

Задание 2

In [ ]:
top = transactions_tbl[['tr_type']].value_counts()[:10].to_frame().reset_index()
top.columns = ['tr_type', 'count']
top_dict = dict(top[['tr_type','count']].values)
descriptions = dict(tr_types_tbl[['tr_type', 'tr_description']].values)
a = []
for tr_type in top_dict:
  a.append([tr_type, top_dict[tr_type], descriptions[tr_type]])
df = pd.DataFrame(a, columns=['tr_type', 'count', 'tr_description'])
pd.options.display.max_colwidth = 200
df

,tr_type,count,tr_description
0,1010,231117,Покупка. POS ТУ СБ РФ
1,2010,151166,Выдача наличных в АТМ Сбербанк России
2,7070,149006,Перевод на карту (с карты) через Мобильный банк (без взимания комиссии с отправителя)
3,1110,137658,Покупка. POS ТУ Россия
4,1030,118975,Оплата услуги. Банкоматы СБ РФ
5,2370,49830,Списание с карты на карту по операции <перевода с карты на карту> через Мобильный банк (без комиссии)
6,7010,28972,Взнос наличных через АТМ (в своем тер.банке)
7,7030,26078,Перевод на карту (с карты) через АТМ (в пределах одного тер.банка)
8,7071,15781,Перевод на карту (с карты) через Мобильный банк (с взиманием комиссии с отправителя)
9,1100,15476,Покупка. ТУ Россия


Задание 3

In [ ]:
client_max= transactions_tbl[transactions_tbl['amount'] > 0].groupby('customer_id').sum()['amount'].idxmax()
client_min = transactions_tbl[transactions_tbl['amount'] < 0].groupby('customer_id').sum()['amount'].idxmin()
clients = transactions_tbl.groupby('customer_id').sum()
print("1.Client_max ", client_max)
print("2.Client_min ", client_min)
print("3.Client_diff", abs(clients.loc[client_max]['amount']))

1.Client_max  70780820
2.Client_min  70780820
3.Client_diff 1837317.9799995997


Задание 4


In [ ]:
grouped_1 = transactions_tbl[transactions_tbl['tr_type'].isin(top['tr_type'])].groupby(['tr_type'])['amount']
print("Mean_tr_type_top_10\n ") 
grouped_1.mean().to_frame()

Mean_tr_type_top_10
 


,amount
tr_type,
1010,-19784.748640
1030,-5320.980222
1100,-44061.827262
1110,-32119.330371
2010,-136077.629325
2370,-205418.249032
7010,276391.789596
7030,86104.332909
7070,65569.831700


In [ ]:
print("\nMedian_tr_type_top_10\n ")
grouped_1.median().to_frame()


Median_tr_type_top_10
 


,amount
tr_type,
1010,-7411.52
1030,-2245.92
1100,-10188.26
1110,-11207.57
2010,-44918.32
2370,-44918.32
7010,112295.79
7030,13951.52
7070,11319.42


In [ ]:
grouped_2 = transactions_tbl[transactions_tbl['customer_id'].isin((client_max, client_min))].groupby(['tr_type'])['amount']
print("Client_all_tr_mean\n ")
grouped_2.mean().to_frame()

Client_all_tr_mean
 


,amount
tr_type,
2010,-2.941056e+06
2330,-2.382398e+06
2370,-2.218505e+06
6110,1.756293e+04
7020,1.465129e+04
7021,2.850067e+04
7030,1.406196e+04
7031,1.921036e+04
7034,1.430648e+04


In [ ]:
print("\nClient_all_tr_median\n ")
grouped_2.median().to_frame()


Client_all_tr_median
 


,amount
tr_type,
2010,-3368873.660
2330,-2245915.770
2370,-2245915.770
6110,10028.010
7020,10319.985
7021,28500.670
7030,8848.910
7031,12992.620
7034,14306.480


Задание 5-8 подготовка

In [ ]:
transactions_tbl = pd.merge(transactions_tbl, gender_train_tbl, how='left')
transactions_tbl = pd.merge(transactions_tbl, tr_mcc_codes_tbl, how='inner')
transactions_tbl = pd.merge(transactions_tbl, tr_types_tbl, how='inner')

Задание 5

In [ ]:
df = transactions_tbl.groupby(['gender'])
gender_1 = transactions_tbl.groupby(['gender']).get_group(1)
gender_0 = transactions_tbl.groupby(['gender']).get_group(0)
gender_1_outcomes = gender_1[gender_1.amount < 0]
gender_0_outcomes = gender_0[gender_0.amount < 0]
print("Mean outcomes difference: ",abs(gender_0_outcomes.amount.mean() - gender_1_outcomes.amount.mean()))
gender_1_incomes = gender_1[gender_1.amount > 0]
gender_0_incomes = gender_0[gender_0.amount > 0]
print("Mean incomes difference: ",abs(gender_0_incomes.amount.mean() - gender_1_incomes.amount.mean()))

Mean outcomes difference:  32718.05492021877
Mean incomes difference:  63366.57104801515


Задание 7

In [ ]:
sum_genders = transactions_tbl[transactions_tbl['amount'] < 0]
sum_genders = sum_genders.groupby(['gender', 'mcc_code', 'mcc_description'])
sum_genders = sum_genders['amount'].sum()
module = abs(sum_genders.get(1) - sum_genders.get(0))
module = module.sort_values(ascending=False)
module = module.head(10)
module.to_frame()

,,amount
mcc_code,mcc_description,
6011,Финансовые институты — снятие наличности автоматически,3.085862e+09
4829,Денежные переводы,2.499153e+09
6010,Финансовые институты — снятие наличности вручную,2.208580e+08
5541,Станции техобслуживания,1.554326e+08
5511,"Легковой и грузовой транспорт — продажа, сервис, ремонт, запчасти и лизинг",7.352877e+07
5812,"Места общественного питания, рестораны",7.037856e+07
5977,Магазины косметики,4.241762e+07
5533,Автозапчасти и аксессуары,4.233808e+07
7995,Транзакции по азартным играм,4.079564e+07


Задание 6


In [ ]:
gender_1_df=gender_1[gender_1.amount>0]
gender_1_max_income=gender_1_df.groupby('tr_type').amount.max()
gender_1_max_income.sort_values(inplace=True)
gender_1_max_income=gender_1_max_income[:10]
gender_1_max_income.to_frame()

,amount
tr_type,
4110,2245.92
2020,2245.92
1010,28803.87
8100,64682.37
6000,79280.83
7041,87590.72
2010,89836.63
2110,112295.79
2370,113194.15


In [ ]:
gender_0_df=gender_0[gender_0.amount>0]
gender_0_max_income=gender_0_df.groupby('tr_type').amount.max()
gender_0_max_income.sort_values(inplace=True)
gender_0_max_income=gender_0_max_income[:10]
gender_0_max_income.to_frame()

,amount
tr_type,
4051,1122.96
4110,2245.92
4210,2245.92
2370,2335.75
4100,5041.86
7075,6737.75
2110,22459.16
1010,24839.83
2010,44918.32


In [ ]:
idx1 = gender_1_max_income.index
idx2 = gender_0_max_income.index
idx=idx1.intersection(idx2)
idx.to_frame(index=False, name='same_tr_types')

,same_tr_types
0,4110
1,1010
2,2010
3,2110
4,2370


Дополнительное задание

In [ ]:
sum_genders = transactions_tbl[transactions_tbl['amount'] < 0]
sum_genders = sum_genders.groupby(['gender', 'mcc_code', 'mcc_description'])
sum_genders = sum_genders['amount'].sum()
module = sum_genders.get(1)/sum_genders.get(0)
module = module.to_frame().reset_index()
module.columns = ['mcc_code', 'mcc_description', 'diff']
bigger = np.ones(len(module.values))
module['bigger outcome belongs to gender'] = bigger.tolist()
module.loc[(module['diff'] < 1), 'bigger outcome belongs to gender'] = 0
module.loc[(module['diff'] < 1), 'diff'] = 1/module['diff']
module = module.sort_values(by='diff', ascending=False)
module.head(10)

,mcc_code,mcc_description,diff,bigger outcome belongs to gender
127,6211,Ценные бумаги: брокеры/дилеры,598.605439,1.0
141,7372,"Программирование, обработка данных, интегрированные системы, дизайн",418.026630,1.0
160,7995,Транзакции по азартным играм,344.079967,1.0
173,8398,Организации благотворительные и общественные службы,295.844430,0.0
43,5193,"Оборудование для выращивания растений, инвентарь для питомников и цветы",103.351765,1.0
167,8062,Больницы,69.686097,0.0
25,5013,Поставщики грузовиков и запчастей,59.702847,1.0
31,5072,Оборудование и сопутствующие материалы для технического обеспечения,56.856720,1.0
26,5039,Строительные материалы — нигде более не классифицированные,50.224801,1.0
143,7395,"Фотостудии, фотолаборатории",30.826054,1.0
